In [1]:
import pooch


url = f"https://github.com/juoceano/geoquimica_marinha/releases/download/"
version = "v0.0.1.dev"

fname = pooch.retrieve(
    url=f"{url}/{version}/Pratica_Campo_Geoquimica_20221.xlsx",
    known_hash="sha256:fd0c72203a143c12f91a0524bd54e884b6b075aae4e06cf9d4c50343310c13f6",
)

In [2]:
import pandas as pd


df = pd.read_excel(fname, sheet_name=0, skiprows=1)

df = df.dropna()
df = df.loc[df["Lat"] != "Lat"]

df[["Lat", "Long"]]

,Lat,Long
0,"27º34'40,4016''","48º31'10,11468''"
2,-27.577101,-48.519441
3,-27.577263,-48.519467
4,-27.577258,-48.519474
5,-27.577112,-48.519085
7,"27º34'40,4976''","48º30'39,83364''"
10,"48°31'2,8506""W","27°35'18,69468""S"
11,"48°31'3,36972""W","27°35'18,69252""S"
12,"48°31'7,73868""W","27°35'15,62964""S"
13,"48°31'9,08976""W","27°35'15,63072""S"


In [3]:
lons = df.loc[10:14]["Lat"].copy()
lats = df.loc[10:14]["Long"].copy()

df["Lat"].loc[10:14] = lats
df["Long"].loc[10:14] = lons

In [4]:
df[["Lat", "Long"]]

,Lat,Long
0,"27º34'40,4016''","48º31'10,11468''"
2,-27.577101,-48.519441
3,-27.577263,-48.519467
4,-27.577258,-48.519474
5,-27.577112,-48.519085
7,"27º34'40,4976''","48º30'39,83364''"
10,"27°35'18,69468""S","48°31'2,8506""W"
11,"27°35'18,69252""S","48°31'3,36972""W"
12,"27°35'15,62964""S","48°31'7,73868""W"
13,"27°35'15,63072""S","48°31'9,08976""W"


In [5]:
import numpy as np
import re

def fix_positions(pos):
    if pos.lower() in ["lat", "long"]:
        return np.NaN
    deg, rest = re.split("º|°", pos)
    mn, *rest = rest.split("'", maxsplit=1)
    rest = [val for val in rest if val][0]
    sec = rest.replace(",", ".").strip("''").strip('"W').strip('"S')
    deg, mn, sec = list(map(float, (deg, mn, sec)))
    return -(deg +  (mn/60) + (sec/60/60))


lons, lats = [], []
for k, row in df.iterrows():
    lon, lat = row["Long"], row["Lat"]
    try:
        lon, lat = list(map(float, (lon, lat)))
        # checar se está tudo no hemisfério Sul e no Oeste.
        lat = -1*np.abs(lat)
        lon = -1*np.abs(lon)
    except ValueError:
        lon, lat = list(map(fix_positions, (lon, lat)))
    lons.append(lon)
    lats.append(lat)

In [6]:
df["Lat"] = lats
df["Long"] = lons

df

,ID,Responsavel,Lat,Long,Ceu,Vento,Observacoes
0,SDD 01,Juliana Leonel,-27.577889,-48.519476,Aberto,Parado,Local próximo à linha d'água. Presença de pneu...
2,SDD 03,Juliana Leonel e Julia Freire,-27.577101,-48.519441,Aberto,Parado,Local Afastado da linha da agua (≈ 20m). Sedim...
3,SDD 04,Juliana Leonel e Julia Freire,-27.577263,-48.519467,Aberto,Parado,Local Afastado da linha da agua (≈ 20m). Sedim...
4,SDD 05,Andressa,-27.577258,-48.519474,Aberto,Parado,"Acabando área de lagunculária, início da veget..."
5,SDD 06,Andressa,-27.577112,-48.519085,Aberto,Parado,"Contínuo na bord, muitas tocas, sedimento comp..."
7,SDD 08,Pedro Alípio,-27.577916,-48.511065,Aberto,Parado,Local próximo à linha delimitadora onde começa...
10,IGT-01,GABRIEL MACHADO,-27.588526,-48.517458,LIMPO,SEM VENTO,LOCAL COBERTO COM VEGETAÇÃO NÃO TÍPICA DE MANG...
11,IGT-02,GABRIEL MACHADO,-27.588526,-48.517603,LIMPO,FRACO,"ÁREA SECA, VEGETAÇÃO SOBRE SEDIMENTO COLETADO;..."
12,IGT-03,GABRIEL MACHADO,-27.587675,-48.518816,LIMPO,FRACO,LOCAL EM FRENTE AO CONDOMÍNIO ITAMBÉ; ODOR CAR...
13,IGT-04,MARIANA ZORZO,-27.587675,-48.519192,LIMPO,FRACO,EM FRENTE AO CORPO DE BOMBEIROS; PRESENÇA DE L...


In [7]:
import folium
import folium.plugins
import geopandas


gdf = geopandas.read_file("./Manguezal_Itacorubi/itacorubi.geojson")
meters = gdf.to_crs(epsg=3395)  # WGS 84 / World Mercator


attr = "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP, UPR-EGP, and the GIS User Community"
tiles = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}"

m = folium.Map(zoom_start=15,
    location=[-27.5822, -48.5161],
    tiles=tiles,
    attr=attr,
)

folium.GeoJson(gdf).add_to(m)
folium.plugins.Fullscreen(force_separate_button=True).add_to(m)

m.fit_bounds(m.get_bounds())

for k, row in df.iterrows():
    lat, lon = row["Lat"], row["Long"]
    popup = row["Responsavel"]
    folium.Marker(location=[lat, lon], popup=popup).add_to(m)

In [8]:
m

In [9]:
df.to_csv("positions.csv", index=False)